In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,71632
2,Huelva,Confirmados PDIA 14 días,7266
3,Huelva,Tasa PDIA 14 días,"1415,9050607011322"
4,Huelva,Confirmados PDIA 7 días,2929
5,Huelva,Tasa PDIA 7 dias,"570,7660229553559"
6,Huelva,Total Confirmados,71866
7,Huelva,Curados,54972
8,Huelva,Fallecidos,439


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  71632.0


/tmp/ipykernel_39388/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  20930.0


/tmp/ipykernel_39388/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_39388/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_39388/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_39388/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 165 personas en los últimos 7 días 

Un positivo PCR cada 66 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,71632.0,7266.0,2929.0,513170.0,570.766023,1415.905061,621.0
Huelva-Costa,41524.0,3768.0,1510.0,289548.0,521.502480,1301.338638,301.0
Sierra de Huelva-Andévalo Central,9210.0,1937.0,853.0,67391.0,1265.747652,2874.271045,186.0
Huelva (capital),20930.0,2192.0,873.0,143837.0,606.937019,1523.947246,157.0
Condado-Campiña,20431.0,1552.0,562.0,156231.0,359.723742,993.400798,132.0
Valverde del Camino,2015.0,381.0,176.0,12750.0,1380.392157,2988.235294,45.0
Lepe,3891.0,235.0,99.0,27880.0,355.093257,842.898135,30.0
Isla Cristina,3896.0,221.0,92.0,21393.0,430.047212,1033.048193,25.0
Cortegana,696.0,203.0,87.0,4602.0,1890.482399,4411.125598,24.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),449.0,128.0,42.0,2327.0,1804.899012,5500.644607,7.0
Jabugo,292.0,101.0,42.0,2260.0,1858.407080,4469.026549,12.0
Cortegana,696.0,203.0,87.0,4602.0,1890.482399,4411.125598,24.0
Rosal de la Frontera,244.0,70.0,30.0,1697.0,1767.825575,4124.926341,8.0
Nerva,648.0,190.0,80.0,5169.0,1547.688141,3675.759334,12.0
Marines (Los),40.0,14.0,9.0,399.0,2255.639098,3508.771930,1.0
Valdelarco,13.0,8.0,6.0,231.0,2597.402597,3463.203463,0.0
Campillo (El),252.0,64.0,28.0,2024.0,1383.399209,3162.055336,7.0
Cumbres Mayores,296.0,54.0,11.0,1749.0,628.930818,3087.478559,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paymogo,120.0,12.0,1.0,1162.0,86.058520,1032.702238,0.0,0.083333
Santa Bárbara de Casa,69.0,7.0,1.0,1043.0,95.877277,671.140940,0.0,0.142857
Fuenteheridos,81.0,20.0,4.0,709.0,564.174894,2820.874471,0.0,0.200000
Cumbres Mayores,296.0,54.0,11.0,1749.0,628.930818,3087.478559,2.0,0.203704
Gibraleón,1499.0,183.0,44.0,12737.0,345.450263,1436.759048,15.0,0.240437
Nava (La),15.0,4.0,1.0,258.0,387.596899,1550.387597,0.0,0.250000
Zalamea la Real,320.0,68.0,19.0,3054.0,622.134905,2226.588081,4.0,0.279412
Higuera de la Sierra,121.0,7.0,2.0,1291.0,154.918668,542.215337,0.0,0.285714
Cartaya,3087.0,246.0,72.0,20083.0,358.512174,1224.916596,19.0,0.292683
